In [1]:
from bs4 import BeautifulSoup as bs
import requests

r = requests.get("https://www.kabum.com.br/hardware/disco-rigido-hd?page_number=1&page_size=100&facet_filters=eyJJbnRlcmZhY2UiOlsiU0FUQSJdfQ==&sort=most_searched")
soup = bs(r.content)

In [2]:
# URL de páginas de produtos
lista_url = [
    # Disco Rígido HD
    'https://www.kabum.com.br/hardware/disco-rigido-hd?page_number=1&page_size=100&facet_filters=eyJJbnRlcmZhY2UiOlsiU0FUQSJdfQ==&sort=most_searched'
]

In [3]:
main = soup.select_one('main.sc-ccc9eb50-13.cWkplc')

articles = []
articles = [article for article in main.select('article.productCard')]

In [11]:
items = []

for article in articles:
    item_url = ('https://www.kabum.com.br/$' + article.select_one('a.productLink')['href'] )
    item_name = article.select_one('h3').text
    item_image = article.select_one('a.productLink > img')['src']
    item_openBox = article.find(class_ = 'openboxTagCard')
    if item_openBox:
        item_openBox = True
    else:
        item_openBox = False
    
    r = requests.get(item_url)
    item_soup = bs(r.content)
    item_price = soup.find(class_ = 'finalPrice').text
    if item_price != None:
        item_price =item_price[3:].split('.')        
        # Preços de mil reais pra cima
        if len(item_price) > 1:
            item_price[1] = item_price[1].split(',')
            formatted_item_price = float(f"{item_price[0]}{item_price[1][0]}.{item_price[1][1]}")

        # Preços abaixo de mil reais
        else:
            item_price[0] = item_price[0].split(',')
            formatted_item_price = float(f"{item_price[0][0]}.{item_price[0][1]}")
            item_price = formatted_item_price
    else:
        # O produto não tem preço
        item_price = ''
    
    items.append({
        'url': item_url,
        'name': item_name,
        'price': item_price,
        'image': item_image,
        'openBox': item_openBox
    })